In [1]:
import requests as rq
import pandas as pd

In [2]:
count = 500

metadatasList = []

headers = {
    "Accept": "application/json"
}

for i in range(int(count/10)):
    url = f"https://data.europa.eu/api/hub/search/search?q=&filter=dataset&includes=id,title.en,description.en,languages,modified,issued,catalog.id,catalog.title,catalog.country.id,distributions.id,distributions.format.label,distributions.format.id,distributions.license,categories.label,publisher&page={i}"
    print(i*10)
    response = rq.get(url, headers=headers)
    data = response.json()
    metadatasList.extend(data["result"]["results"])

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [3]:
dataset = pd.json_normalize(metadatasList)

In [4]:
dataset

,modified,id,categories,issued,distributions,catalog.id,catalog.title.de,catalog.country.id,description.en,title.en,publisher.name,publisher.type,publisher.email,publisher.homepage
0,2021-11-23,025856e5-1a50-4cf1-8091-73ca6dfd9a6b,"[{'label': {'nn': 'Jordbruk, fiskeri, skogbruk...",2021-11-19,"[{'format': {'id': 'WFS_SRVC', 'label': 'WFS'}...",gdi-de,GDI-DE,de,WFS Development Plan ‘Gänberg + Simplified Cha...,WFS Gänberg + Simplified change Gänsberg,NaN,NaN,NaN,NaN
1,2021-11-03,02363749-c46e-4f5c-974b-8272ea30b70f,"[{'label': {'nn': 'Jordbruk, fiskeri, skogbruk...",2021-11-03,"[{'format': {'id': 'WFS_SRVC', 'label': 'WFS'}...",gdi-de,GDI-DE,de,WFS service of the urban development plan “Hag...,WFS INSPIRE BPL Hagenreute I 2nd amendment,NaN,NaN,NaN,NaN
2,2022-06-22,c2f2cc24-1cb8-4b2c-96fb-16037ed8c3a5,"[{'label': {'nn': 'Jordbruk, fiskeri, skogbruk...",2022-06-21,"[{'format': {'id': 'WMS_SRVC', 'label': 'WMS'}...",gdi-de,GDI-DE,de,WMS service of the development plan “Simmetsga...,WMS XPlanung BPL “Simmetsgasse 2nd change”,NaN,NaN,NaN,NaN
3,2018-04-25,de-dwd-mosmix-92004,"[{'label': {'nn': 'Vitskap og teknologi', 'de'...",NaN,NaN,gdi-de,GDI-DE,de,DWD’s fully automatic MOSMIX product optimizes...,Model Output Statistics for WEWAK (92004),NaN,NaN,NaN,NaN
4,2018-04-25,de-dwd-mosmix-72753,"[{'label': {'nn': 'Vitskap og teknologi', 'de'...",NaN,NaN,gdi-de,GDI-DE,de,DWD’s fully automatic MOSMIX product optimizes...,Model Output Statistics for FARGO WSO AP (72753),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-07-25,a24b4856-eac3-4c54-a280-969ea28281d8,"[{'label': {'nn': 'Jordbruk, fiskeri, skogbruk...",2021-05-28,"[{'format': {'id': 'WMS_SRVC', 'label': 'WMS'}...",gdi-de,GDI-DE,de,WMS service of the development plan “Köstlin-/...,WMS XPlanung BPL “Köstlin-/Germersheimer Straß...,NaN,NaN,NaN,NaN
496,2021-11-03,a243041c-e5a8-4ae5-8c80-87a340387a2e,"[{'label': {'nn': 'Jordbruk, fiskeri, skogbruk...",2021-11-03,"[{'format': {'id': 'WMS_SRVC', 'label': 'WMS'}...",gdi-de,GDI-DE,de,WMS service of the urban development plan “Bei...,WMS INSPIRE BPL At the village — outdoor set,NaN,NaN,NaN,NaN
497,2024-06-06,a24a966e-ce4d-4e1f-9240-5fad52a1f716,NaN,2020-12-31,"[{'format': {'id': 'WMS_SRVC', 'label': 'WMS'}...",gdi-de,GDI-DE,de,WMS service of the development plan “Young Vin...,WMS XPlanung BPL “Young Vineyards and Industre...,NaN,NaN,NaN,NaN
498,2021-07-18,a22e3d9b-11d0-430e-9b8d-9e110563f13f,"[{'label': {'nn': 'Jordbruk, fiskeri, skogbruk...",2021-01-13,"[{'format': {'id': 'WMS_SRVC', 'label': 'WMS'}...",gdi-de,GDI-DE,de,WMS service of the development plan “Industria...,WMS XPlanung BPL “Industrial Area East 2nd Ame...,NaN,NaN,NaN,NaN


In [5]:
dataset.columns

Index(['modified', 'id', 'categories', 'issued', 'distributions', 'catalog.id',
       'catalog.title.de', 'catalog.country.id', 'description.en', 'title.en',
       'publisher.name', 'publisher.type', 'publisher.email',
       'publisher.homepage'],
      dtype='object')

In [6]:
dataset["categories"].fillna("", inplace=True)

C:\Users\Ramil\AppData\Local\Temp\ipykernel_1480\1089690094.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["categories"].fillna("", inplace=True)


In [7]:
dataset.loc[:, "category"] = dataset.loc[:, "categories"].apply(lambda x: x[0]['label']['en'] if x != '' else None)

In [8]:
dataset = dataset.drop(columns=["modified", "issued", "distributions", "categories"])

In [9]:
dataset

,id,catalog.id,catalog.title.de,catalog.country.id,description.en,title.en,publisher.name,publisher.type,publisher.email,publisher.homepage,category
0,025856e5-1a50-4cf1-8091-73ca6dfd9a6b,gdi-de,GDI-DE,de,WFS Development Plan ‘Gänberg + Simplified Cha...,WFS Gänberg + Simplified change Gänsberg,NaN,NaN,NaN,NaN,"Agriculture, fisheries, forestry and food"
1,02363749-c46e-4f5c-974b-8272ea30b70f,gdi-de,GDI-DE,de,WFS service of the urban development plan “Hag...,WFS INSPIRE BPL Hagenreute I 2nd amendment,NaN,NaN,NaN,NaN,"Agriculture, fisheries, forestry and food"
2,c2f2cc24-1cb8-4b2c-96fb-16037ed8c3a5,gdi-de,GDI-DE,de,WMS service of the development plan “Simmetsga...,WMS XPlanung BPL “Simmetsgasse 2nd change”,NaN,NaN,NaN,NaN,"Agriculture, fisheries, forestry and food"
3,de-dwd-mosmix-92004,gdi-de,GDI-DE,de,DWD’s fully automatic MOSMIX product optimizes...,Model Output Statistics for WEWAK (92004),NaN,NaN,NaN,NaN,Science and technology
4,de-dwd-mosmix-72753,gdi-de,GDI-DE,de,DWD’s fully automatic MOSMIX product optimizes...,Model Output Statistics for FARGO WSO AP (72753),NaN,NaN,NaN,NaN,Science and technology
...,...,...,...,...,...,...,...,...,...,...,...
495,a24b4856-eac3-4c54-a280-969ea28281d8,gdi-de,GDI-DE,de,WMS service of the development plan “Köstlin-/...,WMS XPlanung BPL “Köstlin-/Germersheimer Straß...,NaN,NaN,NaN,NaN,"Agriculture, fisheries, forestry and food"
496,a243041c-e5a8-4ae5-8c80-87a340387a2e,gdi-de,GDI-DE,de,WMS service of the urban development plan “Bei...,WMS INSPIRE BPL At the village — outdoor set,NaN,NaN,NaN,NaN,"Agriculture, fisheries, forestry and food"
497,a24a966e-ce4d-4e1f-9240-5fad52a1f716,gdi-de,GDI-DE,de,WMS service of the development plan “Young Vin...,WMS XPlanung BPL “Young Vineyards and Industre...,NaN,NaN,NaN,NaN,None
498,a22e3d9b-11d0-430e-9b8d-9e110563f13f,gdi-de,GDI-DE,de,WMS service of the development plan “Industria...,WMS XPlanung BPL “Industrial Area East 2nd Ame...,NaN,NaN,NaN,NaN,"Agriculture, fisheries, forestry and food"


In [10]:
dataset["description.en"]

0      WFS Development Plan ‘Gänberg + Simplified Cha...
1      WFS service of the urban development plan “Hag...
2      WMS service of the development plan “Simmetsga...
3      DWD’s fully automatic MOSMIX product optimizes...
4      DWD’s fully automatic MOSMIX product optimizes...
                             ...                        
495    WMS service of the development plan “Köstlin-/...
496    WMS service of the urban development plan “Bei...
497    WMS service of the development plan “Young Vin...
498    WMS service of the development plan “Industria...
499    WMS service of the development plan "Wiesenäck...
Name: description.en, Length: 500, dtype: object

In [11]:
import spacy as sp

In [12]:
nlp = sp.load("en_core_web_sm")

In [13]:
doc = nlp(dataset["description.en"][0])

In [14]:
nlp(dataset["description.en"][0]).similarity(nlp(dataset["description.en"][1]))

C:\Users\Ramil\AppData\Local\Temp\ipykernel_1480\889232210.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp(dataset["description.en"][0]).similarity(nlp(dataset["description.en"][1]))


0.7206575274467468

In [15]:
dataset["description.en"].fillna("")

0      WFS Development Plan ‘Gänberg + Simplified Cha...
1      WFS service of the urban development plan “Hag...
2      WMS service of the development plan “Simmetsga...
3      DWD’s fully automatic MOSMIX product optimizes...
4      DWD’s fully automatic MOSMIX product optimizes...
                             ...                        
495    WMS service of the development plan “Köstlin-/...
496    WMS service of the urban development plan “Bei...
497    WMS service of the development plan “Young Vin...
498    WMS service of the development plan “Industria...
499    WMS service of the development plan "Wiesenäck...
Name: description.en, Length: 500, dtype: object

In [16]:
dataset["description.en"][16]

'Description of INSPIRE Download Service (predefined Atom): Development plan “Special Area Photovoltaics” of the municipality of Beilingen – The link(s) for downloading the data records is/are generated dynamically from Get Map Calling a WMS Interface'

In [17]:
pr_count = len(dataset["description.en"])

In [18]:
pr_count

500

In [19]:
s = [[-1 for _ in range(pr_count)] for _ in range(pr_count)]

In [20]:
for i in range(10):
    for j in range(i+1, 10):
        s[i][j] = nlp(dataset["description.en"][i]).similarity(nlp(dataset["description.en"][j]))
        print(i, j)

C:\Users\Ramil\AppData\Local\Temp\ipykernel_1480\3943533261.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  s[i][j] = nlp(dataset["description.en"][i]).similarity(nlp(dataset["description.en"][j]))


0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 4
3 5
3 6
3 7
3 8
3 9
4 5
4 6
4 7
4 8
4 9
5 6
5 7
5 8
5 9
6 7
6 8
6 9
7 8
7 9
8 9


In [21]:
import numpy as np

In [22]:
np.argsort(s[0])

array([496, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492,
       493, 494, 499, 498,  27,  28,  29,  30, 497,  16,  15, 480,  31,
        10,  11,  12,  13,  14, 495,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [23]:
dataset["description.en"][0]

'WFS Development Plan ‘Gänberg + Simplified Change Gänsberg’ of the city of Herrenberg.'

In [24]:
dataset["description.en"][158]

'WFS service of the urban development plan “Ledigsberg — Expansion” of the city of Eberbach, transformed according to INSPIRE, based on an XPlanung dataset in version 5.0.'

In [25]:
dataset.columns

Index(['id', 'catalog.id', 'catalog.title.de', 'catalog.country.id',
       'description.en', 'title.en', 'publisher.name', 'publisher.type',
       'publisher.email', 'publisher.homepage', 'category'],
      dtype='object')

In [26]:
columns = ["title.en", "description.en", "category"]

In [27]:
fs = np.array([[-1 for _ in range(pr_count)] for _ in range(pr_count)], dtype='d')

In [ ]:

for i in range(pr_count):
    for j in range(i+1, pr_count):
        sim = 0
        for column in columns:
            t1, t2 = dataset[column][i], dataset[column][j]
            s = 0
            if t1 and t2:
                s = nlp(t1).similarity(nlp(t2))
            sim += s*s
        fs[i][j] = np.sqrt(sim)
        print(i, j)

C:\Users\Ramil\AppData\Local\Temp\ipykernel_1480\4012403534.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  s = nlp(dataset[column][i]).similarity(nlp(dataset[column][j]))


0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'NoneType'>

In [ ]:
np.save("similarity.npy", fs)

In [ ]:
ls = np.load("similarity.npy")

In [ ]:
ls

array([[-1.        ,  1.39007509,  1.31678683, ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        ,  1.35572857, ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       ...,
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ]])

In [ ]:
fs

array([[-1.        ,  1.39007509,  1.31678683, ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        ,  1.35572857, ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       ...,
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ]])

In [ ]:
dataset.loc[0]

id                                 025856e5-1a50-4cf1-8091-73ca6dfd9a6b
catalog.id                                                       gdi-de
catalog.title.de                                                 GDI-DE
catalog.country.id                                                   de
description.en        WFS Development Plan ‘Gänberg + Simplified Cha...
title.en                       WFS Gänberg + Simplified change Gänsberg
publisher.name                                                      NaN
publisher.type                                                      NaN
publisher.email                                                     NaN
publisher.homepage                                                  NaN
category                      Agriculture, fisheries, forestry and food
Name: 0, dtype: object

In [ ]:
sample_count = 30
survey = []
for i in range(0, pr_count, int(pr_count/sample_count)):
    main_dataset = dataset.loc[i]
    recommended_datasets = dataset.loc[np.argsort(fs[i])[-5:]].to_dict('records')
    survey.append((main_dataset, recommended_datasets))


In [ ]:
survey

[(id                                 025856e5-1a50-4cf1-8091-73ca6dfd9a6b
  catalog.id                                                       gdi-de
  catalog.title.de                                                 GDI-DE
  catalog.country.id                                                   de
  description.en        WFS Development Plan ‘Gänberg + Simplified Cha...
  title.en                       WFS Gänberg + Simplified change Gänsberg
  publisher.name                                                      NaN
  publisher.type                                                      NaN
  publisher.email                                                     NaN
  publisher.homepage                                                  NaN
  category                      Agriculture, fisheries, forestry and food
  Name: 0, dtype: object,
  [{'id': 'de-dwd-mosmix-72503',
    'catalog.id': 'gdi-de',
    'catalog.title.de': 'GDI-DE',
    'catalog.country.id': 'de',
    'description.en': 'DWD’s full

In [ ]:
survey[3][0]["description.en"]

'The ISID34 TTAAii Data Designators decode as: T1 (I): Observational data (Binary coded) - BUFR T1T2 (IS): Surface/sea level T1T2A1 (ISI): Intermediate synoptic observations from fixed land stations A2 (D): 90°E - 0° northern hemisphere (Remarks from Volume-C: NATIONAL AUTOMATIC SYNOP)'

In [ ]:
survey[3][1][3]

{'id': 'a236eba9-5734-4502-b1d4-60b72e03a7ee',
 'catalog.id': 'gdi-de',
 'catalog.title.de': 'GDI-DE',
 'catalog.country.id': 'de',
 'description.en': 'WMS service of the development plan "Wiesenäcker I - 1. Änderung" of the city of Ditzingen from XPlanung 5.0. Description: Wiesenäcker I - 1st amendment.',
 'title.en': 'WMS XPlanung BPL "Wiesenäcker I - 1st Amendment"',
 'publisher.name': nan,
 'publisher.type': nan,
 'publisher.email': nan,
 'publisher.homepage': nan,
 'category': None}

In [ ]:
import jinja2 as jj

In [ ]:
fsl = jj.FileSystemLoader(searchpath="templates")

In [ ]:
env = jj.Environment(loader=fsl)

In [ ]:
fsl.list_templates()

['template.jinja']

In [ ]:
template = env.get_template("template.jinja")

In [ ]:
recs = []

In [ ]:
survey[0][0]["description.en"]

'WFS Development Plan ‘Gänberg + Simplified Change Gänsberg’ of the city of Herrenberg.'

In [ ]:
indexHtml = template.render(survey = survey).encode( "utf-8" )

In [ ]:
file=open("index.html", "wb")
file.write(indexHtml)
file.close()

In [ ]:
survey[2][0]["title.en"]

'Model Output Statistics for SAMARKAND (38696)'

In [ ]:
survey[2][1][0]["title.en"]

'Inspire Download Service (predefined ATOM) for Dataset Special Area Photovoltaics'